# Preliminary notes


### 1. Do not be afraid of python

WannierBerri is provided as a python module. This means that unlike QE or Wannier90, there is no distinction between "program" and input files. All input parameters are provided as arguments to calls of functions and class constructors. (While Wannier function's data is read from the files provided by Wanier90. This may sound (and look) scary to those who never used python (or never coded at all). But please, do not stand up and leave the classroom :)

At first, you may use the templates given in the tutorials and modify only those parameters that you understand. The more you learn python, the more advantages you may get from the code. 

In fact, the border between "using" and "coding" becomes smeared. In one of the advanced tutorials, it is demonstrated how one can compute quantities that are not yet implemented in WannierBerri, without actually modifying the source code of the package itself.

### 2. Jupyter Notebooks

In this tutorial, we work with a Jupyter notebook. This is a convenient interactive environment that allows you to play with the code. For production runs, you may want to combine your workflow in one python script and execute it on a (super)computer


### 3. Documentation 

A detailed description of parameters is in https://wannier-berri.org/documentation.html

Note : also check the parameters of a parent class!

### 4. New interface

Those who are already familiar with WB, know that one should use [integrate()](https://wannier-berri.org/docs/oldAPI.html#wannierberri.integrate) or [tabulate()](https://wannier-berri.org/docs/oldAPI.html#wannierberri.tabulate) functions to run calculations. In the new version, they are substituted by  the [run()](https://wannier-berri.org/docs/run.html). This interface looks to be less laconic for basic calculations, but for advanced uses, it provides a powerful tool to pass different parameters to different calculations.

### 5. Feedback 

If you have questions or comments after this tutorial is over, use the following ways of communication:

* Discussions or Issues on GitHub : https://github.com/wannier-berri/wannier-berri/discussions

* mailing list : https://physik.lists.uzh.ch/sympa/info/wannier-berri




# Tutorial on Wannier Berri : interpolating bands, Berry curvatures, and integrating them

In this tutorial we will see how to compute band energies, Berry curvature, spins and anomalous Hall conductivity  using WannierBerri code and further integrate them over the Brillouin zone to obtain Anomalous Hall conductivity, and other quantities of interest.

## Preparation of a calculation:

* import the needed modules

* initialize a [Parallel() or Serial()](https://wannier-berri.org/docs/parallel.html) class 


In [ ]:
# Preliminary 

# Set environment variables - not mandatory but recommended if you use Parallel()
#import os
#os.environ['OPENBLAS_NUM_THREADS'] = '1' 
#os.environ['MKL_NUM_THREADS'] = '1'


import wannierberri as wberri
print (f"Using WannierBerri version {wberri.__version__}")
import numpy as np
import scipy
import matplotlib.pyplot as plt
from termcolor import cprint

#  This block is needed if you run this cell for a second time
#  because one cannot initiate two parallel environments at a time
try:
    parallel.shutdown()   
except NameError:
    pass

# Chiose one of the options:

parallel = wberri.Parallel(num_cpus=2)
#parallel = wberri.Parallel()  # automatic detection 
#parallel = wberri.Serial()



## Reading the system

Now we need to define the system that we are working on. Wannierberri can equally work with Wannier functions obtained by Wannier90 or other code (e.g. ASE or FPLO), as well as tight-binding models. This is done by constructing a [System()](https://wannier-berri.org/docs/system.html) class or one of its subclasses. Below, an example for Wannier90 is given; in advanced tutorials, some tight-binding models are also used.

In [ ]:
# Importing data from wannier90

system=wberri.System_w90(
                        seedname='w90_files/Fe',
                        berry=True,   # needed to calculate "external terms" of Berry connection or curvature , reads ".mmn" file
                        spin = True , # needed for spin properties, reads ".spn" file
                        use_wcc_phase=True  # include the fphase actors e^(i*k*r_a) in the Bloch basis state definition
                        )

generators=["Inversion","C4z","TimeReversal*C2x"]
system.set_symmetry(generators)


## Interpolation on a path

In [ ]:
# Evaluate bands, Berry curvature, and spin along a path GHPNG


# all kpoints given in reduced coordinates
path=wberri.Path(system,
                 k_nodes=[
        [0.0000, 0.0000, 0.0000 ],   #  G 
        [0.500 ,-0.5000, -0.5000],   #  H 
        [0.7500, 0.2500, -0.2500],   #  P 
        [0.5000, 0.0000, -0.5000],   #  N 
        [0.0000, 0.0000, 0.000  ] ] , #  G 
                 labels=["G","H","P","N","G"],
                 length=200 )   # length [ Ang] ~= 2*pi/dk

# uncomment some of these lines to see what path you have generated

#print (path)

#print (path.getKline())

#for K in path.get_K_list():
#    print (K)

#print (np.array([K.K for K in path.get_K_list()]))



## Running a calculation: calculators and results

The calculation is called using the function [`run()`](https://wannier-berri.org/docs/run.html) : 

```
result=wberri.run(system, 
                  grid=path, 
                  calculators = {"key" : calculator},
                  parallel = parallel, 
                  print_Kpoints = False)
```
Here, apart from the already known ingredients, we need to define objects of some subclass of [`Calculator`](https://wannier-berri.org/docs/calculators.html#wannierberri.calculators.Calculator). A Calculator is a  callable object which returns some [`Result`](https://wannier-berri.org/docs/result.html). If you code another calculator, you can calculate other things using the machinery of WannierBerri, but that is not a part of this basic tutorial. 

Further, results are packed into [`ResultDict`](https://wannier-berri.org/docs/result.html#wannierberri.result.ResultDict) and returned. 


## Tabulating Berry curvature and spin

To run a tabulation one needs to compose a dictionary, where keys are any arbitrary strings to label further tabulation results, and the values are subclasses of [`Tabulator`](https://wannier-berri.org/docs/calculators.html#wannierberri.calculators.tabulate.Tabulator)

Next, we pack them into another class called [`TabulatorAll`](https://wannier-berri.org/docs/calculators.html#wannierberri.calculators.TabulatorAll), which represents a complex [`Calculator`](https://wannier-berri.org/docs/calculators.html#wannierberri.calculators.Calculator)

In [ ]:

tabulators = { "Energy": wberri.calculators.tabulate.Energy(),
               "berry" : wberri.calculators.tabulate.BerryCurvature(),
             }

tab_all_path = wberri.calculators.TabulatorAll(
                    tabulators,
                    ibands = np.arange(0,18),
                    mode = "path"
                        )

## Running a calculation

Now run the calculation using the function [`run()`](https://wannier-berri.org/docs/run.html) . This will return an object of class [`ResultDict()`](https://wannier-berri.org/docs/result.html#wannierberri.result.ResultDict)
This object contains all results of the calculations, but in this case we have only one, which is marked `"tabulate"`


In [ ]:
result=wberri.run(system, 
                  grid=path, 
                  calculators = {"tabulate" : tab_all_path},
                  parallel = parallel, 
                  print_Kpoints = False)

print (result.results)
path_result = result.results["tabulate"]


## Plotting the results

The [`TABresult`](https://wannier-berri.org/docs/result.html#wannierberri.result.TABresult) object already provides methods to plot the results. (As well as one can extract the data and plot them by other means). Below let's plot the interpolated bands and compare with those obtained in QE. (file "bands/Fe_bands_pw.dat" is already provided)


In [ ]:
# plot the bands and compare with QuantumEspresso
EF = 12.6
path_result = result.results["tabulate"]
# Import the pre-computed bands from quantum espresso
A = np.loadtxt(open("bands/Fe_bands_pw.dat","r"))
bohr_ang = scipy.constants.physical_constants['Bohr radius'][0] / 1e-10
alatt = 5.4235* bohr_ang
A[:,0]*= 2*np.pi/alatt
A[:,1]-=EF
# plot it as dots
plt.scatter (A[:,0],A[:,1],s=5,label = "QE")


path_result.plot_path_fat( path,
              quantity=None,
              save_file="Fe_bands+QE.pdf",
              Eshift=EF,
              Emin=-10,  Emax=50,
              iband=None,
              mode="fatband",
              fatfactor=20,
              cut_k=False, 
              close_fig=False,
              show_fig=False,
              label = "WB"
              )


plt.legend()
plt.show()
plt.close()


In [ ]:
# plot the bands and compare with wannier90
A = np.loadtxt(open("bands/Fe_bands_w90.dat","r"))
plt.scatter (A[:,0],A[:,1],s=5,label = "W90")

path_result.plot_path_fat( path,
              quantity=None,
              save_file="Fe_bands+w90.pdf",
              Eshift=0,
              Emin=4,  Emax=20,
              iband=None,
              mode="fatband",
              fatfactor=20,
              cut_k=False, 
              close_fig=False,
              show_fig=False,
              label = "WB"
              )

plt.legend()

plt.show()
plt.close()

In [ ]:
# plot the Berry curvature
path_result.plot_path_fat( path,
              quantity='berry',
              component='z',
              save_file="Fe_bands+berry.pdf",
              Eshift=0,
              Emin=4,  Emax=20,
              iband=None,
              mode="fatband",
              fatfactor=4,
              cut_k=False, 
              close_fig=True,
              show_fig=True
              )

# The size of the dots corresponds to the magnitude of BC on a logarithmic scale

## Problem 1:

* modify the path
* plot the "z" component of spin along it (without . 
* do **not** plot QE or W90 bands in this case

Hint : look here for a proper Calculator https://wannier-berri.org/docs/calculators.html#tabulating


In [ ]:
# put the necessary code here






## Get the data and do whatever you want

In [ ]:
k=path.getKline()
E=path_result.get_data(quantity='Energy',iband=(10,11))
curv=path_result.get_data(quantity='berry',iband=(10,11),component="z")
print (k.shape, E.shape, curv.shape)

## Calaculation on a 3D grid

Now let's investigate how Berry curvature behaves in the 3D  Brillouin zone. For that we need to set a grid, which can be done in several ways, see input parameters [here](file:///home/stepan/github/wannier-berri-org/html/docs/grid.html)

Most important to recall, is that in WB one sets two grids : the FFT grid and the K-grid (NKdiv). this is important for running the calculation. However, the final; result depends only on their product. 

In [ ]:
# Set a grid 
grid = wberri.Grid(system, length=50 )   # length [ Ang] ~= 2*pi/dk
#grid = wberri.Grid(system, NK=[24,24,24], NKFFT=4)
#grid = wberri.Grid(system, NKdiv=6, NKFFT=4)





### We can use the same tabulators, but now we pack them into TabulatorAll in "grid" mode

In [ ]:
tabulators = { "Energy": wberri.calculators.tabulate.Energy(),
               "berry" : wberri.calculators.tabulate.BerryCurvature(),
             }

tab_all_grid = wberri.calculators.TabulatorAll(
                    tabulators,
                    ibands = np.arange(0,18),
                    mode = "grid"
                        )

### And we run the calculation in the same way

In [ ]:
result=wberri.run(system, 
                  grid=grid, 
                  calculators = {"tabulate" : tab_all_grid},
                  parallel = parallel, 
                  print_Kpoints = True)

print (result.results)
grid_result = result.results["tabulate"]

In [ ]:
# Now we got some .frmsf files
!ls -al *.frmsf
#!rm *.frmsf

In [ ]:
# let's look at them using the Fermisurfer! (https://fermisurfer.osdn.jp/)
#> fermisurfer Fe_berry_z.frmsf

## Analyze the tabulated data

In [ ]:
# You may get the data as numpy arrays via:
Energy = grid_result.get_data(iband=5, quantity='Energy')
berry  = grid_result.get_data(iband=5, quantity='berry',component='z')
print(berry.shape,Energy.shape)

## Problem 2 : 

### fill the missing parts and evaluate the Berry curvature summed over all states below EF = 12.6 eV. Plot in on a plane k3=const (in reduced coordinates)

In [ ]:
# example : find the total Berry curvature of occupied states
# on the plane (k1,k2), k3=const (in reduced coordinates)
Berry_occ = 0
k3 = 9
EF=12.4
for ib in range(18):
    Energy = grid_result.get_data(iband=ib, quantity='Energy')[:,:,0]
    berry  = grid_result.get_data(iband=ib, quantity='berry',component='z')[:,:,0]
    berry [Energy>EF] = 0
    Berry_occ += berry
    plt.contour(Energy,levels = [EF],linewidths=0.5,colors='black')
shw = plt.imshow(Berry_occ,vmin=-10,vmax=10,cmap="jet")
bar = plt.colorbar(shw)

## Integration on a grid: anomalous Hall conductivity

Now, after we saw that the Berry curvature changes rapidly in the k-space, we understand that to get the precise value of AHC (\ref{eq:ahc}) defined as a Fermi-sea integral of Berry curvature

\begin{equation}
\sigma^{\rm AHC}_{xy} = -\frac{e^2}{\hbar} \sum_n^{\rm occ} \int\frac{d\mathbf{k}}{(2\pi)^3} \Omega^n_\gamma
\label{eq:ahc}\tag{1}
\end{equation}

we need a very dense grid. The calculation is done again, by using the calculators. AHC may be viewed as a function of the Fermi level. Such calculators are called [StaticCalculator](https://wannier-berri.org/docs/calculators.html#static-dependent-only-on-fermi-level) , because the corresponding effects can be measured in static fields. (as opposed to dynamic calculators, see below).

In [ ]:
calculators = {}
Efermi = np.linspace(12,13,101)
omega = np.linspace(0,1.,101)
# Set a grid 
grid = wberri.Grid(system, length=50 )   # length [ Ang] ~= 2*pi/dk

calculators ["ahc"] = wberri.calculators.static.AHC(Efermi=Efermi)

result_run = wberri.run(system,
            grid=grid,
            calculators = calculators,
            parallel=parallel,
            adpt_num_iter=5,
            fout_name='Fe',
            restart=False,
            file_Klist="Klist_ahc.pickle"  # needed to restart a calculation in future
            )



In [ ]:
!ls

In [ ]:
#plot results from different iterations
for i in range(5):
    a = np.loadtxt(f"Fe-ahc_iter-{i:04d}.dat")
    plt.plot(a[:,0],a[:,3],label = f"iteration-{i}")
#plt.ylim(-1000,1000)
plt.legend()
plt.show()

In [ ]:
result_run = wberri.run(system,
            grid=grid,
            calculators = calculators,
            parallel=parallel,
            adpt_num_iter=10,
            fout_name='Fe',
            restart=True,
            file_Klist="Klist_ahc.pickle"  # needed to restart a calculation
            )

In [ ]:
#plot results from different iterations
for i in range(10,15):
    a = np.loadtxt(f"Fe-ahc_iter-{i:04d}.dat")
    ef = a[:,0] 
    ahc_xy = a[:,3]
    # alternatively:
    #res = np.load(f"Fe-ahc_iter-{i:04d}.npz")
    #print (list(res.keys()))
    #ef = res["Energies_0"]
    #ahc_xy = res["data"][:,2]
    plt.plot(ef,ahc_xy,label = f"iteration-{i}")
#plt.ylim(-1000,1000)
plt.legend()
plt.show()

## Problem 3 : 
### start from a denser grid (length=100 or 200) and do the integration again with 20 iterations. Plot the results

In [ ]:
# insert the needed code below




## Tetrahedron method

In [ ]:
calculators = {}
Efermi = np.linspace(12,13,101)
omega = np.linspace(0,1.,101)
# Set a grid 
grid = wberri.Grid(system, length=50 )   # length [ Ang] ~= 2*pi/dk

calculators ["dos_notetra"] = wberri.calculators.static.DOS(Efermi=Efermi,tetra=False)
calculators ["dos_tetra"] = wberri.calculators.static.DOS(Efermi=Efermi,tetra=True)

result_run = wberri.run(system,
            grid=grid,
            calculators = calculators,
            parallel=parallel,
            adpt_num_iter=0,
            fout_name='Fe',
            suffix = "run2",
            restart=False,
            print_Kpoints=False
            )

a = np.loadtxt(f"Fe-dos_notetra-run2_iter-0000.dat")
plt.plot(a[:,0],a[:,1],label = f"no tetra")
a = np.loadtxt(f"Fe-dos_tetra-run2_iter-0000.dat")
plt.plot(a[:,0],a[:,1],label = f"tetra")
plt.legend()

In [ ]:
!ls


## Optical conductivity

In [ ]:
calculators = {}
Efermi = np.linspace(12,13,101)
omega = np.linspace(0,1.,101)
# Set a grid 
grid = wberri.Grid(system, length=50 )   # length [ Ang] ~= 2*pi/dk

calculators["opt_conductivity"] = wberri.calculators.dynamic.OpticalConductivity(
                            Efermi=Efermi,omega=omega)


result_run_opt = wberri.run(system,
            grid=grid,
            calculators = calculators,
            parallel=parallel,
            adpt_num_iter=0,
            fout_name='Fe',
            suffix = "run3",
            restart=False,
            )

In [ ]:
#plot results from new iterations
res = result_run_opt.results["opt_conductivity"]
print (res.data.shape)
print (res.Energies[0]) # Efermi
print (res.Energies[1]) # omega

# plot at fixed omega
iw = 10
plt.plot(res.Energies[0],data[:,iw,2,2].imag)
plt.show()

# plot at fixed Efermi
ief = 20
plt.plot(res.Energies[1],data[ief,:,2,2].imag)
plt.show()


In [ ]:
!ls

## All in one

In [ ]:
calculators = {}
Efermi = np.linspace(12,13,101)
omega = np.linspace(0,1.,101)
# Set a grid 
grid = wberri.Grid(system, length=50 )   # length [ Ang] ~= 2*pi/dk

calculators ["ahc_notetra"] = wberri.calculators.static.AHC(Efermi=Efermi,tetra=False)
calculators ["ahc_tetra"] = wberri.calculators.static.AHC(Efermi=Efermi,tetra=True)
calculators ["tabulate"] = wberri.calculators.TabulatorAll({
                            "Energy":wberri.calculators.tabulate.Energy(),
                            "berry":wberri.calculators.tabulate.BerryCurvature(),
                                  },   
                            ibands = np.arange(4,10))
calculators["opt_conductivity"] = wberri.calculators.dynamic.OpticalConductivity(
                            Efermi=Efermi,omega=omega)


result_run = wberri.run(system,
            grid=grid,
            calculators = calculators,
            parallel=parallel,
            adpt_num_iter=0,
            fout_name='Fe',
            suffix = "run",
            restart=False,
            )